In [ ]:
import tensorflow as tf

In [ ]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import cv2
from imutils import paths
import os
import time
import random
from keras.preprocessing.image import img_to_array

data=[]
labels=[]
i=1
d=[]

In [ ]:
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/Datasets/dataset1")))
print(imagePaths)
for imagePath in imagePaths:
  if i%50==0:
    print(i)
  i+=1
  image = cv2.imread(imagePath)
  image = cv2.resize(image,(50, 50))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  image = img_to_array(image)
  label = imagePath.split(os.path.sep)[-2]
  if label=="with_mask":
    label=[1,0,0]
    d.append((image,label))
  elif label=="without_mask":
    label=[0,1,0]
    d.append((image,label))
  elif label=="Masks2":
    label=[0,0,1]
    d.append((image,label))

In [ ]:
random.shuffle(d)

In [ ]:
import pickle
with open("/content/drive/MyDrive/Datasets/mask12.txt", "rb") as fp:
  d = pickle.load(fp)

In [ ]:
for i in d:
  data.append(i[0])
  labels.append(i[1])

In [ ]:
len(data)
print(labels.count([0,0,1]))

732


In [ ]:
import numpy as np
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
train_X = data
train_y = labels
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(data)*VAL_PCT)
print(val_size)

train_X = data[:2*-val_size]
train_y = labels[:2*-val_size]

test_X = data[2*-val_size:-val_size]
test_y = labels[2*-val_size:-val_size]

test_X1 = data[-val_size:]
test_y1 = labels[-val_size:]


504


In [ ]:
test_y.count([0,0,1])

73

In [ ]:
EPOCHS = 29
BATCH_SIZE = 128
steps_per_epoch = len(train_X)//BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)  

Steps per epoch:  31


In [ ]:
with tf.device('/device:GPU:0'):    
      model = tf.keras.Sequential(
      [
        
        tf.keras.layers.Conv2D(32, (5, 5), input_shape = (50, 50, 1), activation = 'relu'), # no bias necessary before batch norm
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
        
        tf.keras.layers.Conv2D(64, (5, 5), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
        
        tf.keras.layers.Conv2D(128, (5, 5), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units = 512, activation = 'relu'),
        
        tf.keras.layers.Dense(3, activation='softmax')
      ])

      model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import *
mc=ModelCheckpoint('plant_gpu.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1)  
start_time=time.time()
history = model.fit(train_X,train_y, steps_per_epoch=steps_per_epoch, epochs=EPOCHS,validation_data=(test_X,test_y),batch_size=BATCH_SIZE,callbacks=[mc])
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from tensorflow.keras.models import load_model
model=load_model('plant_gpu.h5')
c=0
t=0
for i in range(len(test_X1)):
  test_image = np.expand_dims(test_X1[i], axis = 0)
  result=np.argmax(model.predict(test_image),axis=1)
  if result[0]==0 and test_y1[i][0]==1:
    c+=1
  if result[0]==1 and test_y1[i][1]==1:
    c+=1
  if result[0]==2 and test_y1[i][2]==1:
    c+=1
  t+=1

In [ ]:
c/t

0.9444444444444444

Storing the shuffle data using pickle.

In [ ]:
with open("test12.txt", "wb") as fp:   
   pickle.dump(d, fp)